In [1]:
import pandas as pd

In [2]:
uniprot_mapping = pd.read_csv('raw_data/uniprot-yourlist_M201910086746803381A1F0E0DB47453E0216320D0044DCI+active_yes.tab', sep='\t')
hippie = pd.read_csv('raw_data/hippie_current.txt', sep='\t', names=['A', 'Aa', 'B', 'Bb', 'score', 'evidence'])
location_data = pd.read_excel('raw_data/41467_2015_BFncomms8866_MOESM610_ESM.xlsx', 1)

In [3]:
display(uniprot_mapping[:3])
display(location_data[:3])
display(hippie[:3])

,Entry,Entry name,Protein names,Gene names
0,P04217,A1BG_HUMAN,Alpha-1B-glycoprotein (Alpha-1-B glycoprotein),A1BG
1,Q9NQ94,A1CF_HUMAN,APOBEC1 complementation factor (APOBEC1-stimul...,A1CF ACF ASP
2,P01023,A2MG_HUMAN,Alpha-2-macroglobulin (Alpha-2-M) (C3 and PZP-...,A2M CPAMD5 FWP007


,ApprovedSymbol,HGNCID,ApprovedName,UniProtAC,UniProt_SL,HPRD_SL,HPRD_SL.PMID,LocTree3_Localization,LocTree3_Score,TMHsPredicted_PolyPhobius,Consensus_SL,6class_consensus_SL (Fig. 1),Taxon.Wagner,Taxon.Dollo,Taxon.Tautz2013
0,A1BG,HGNC:5,alpha-1-B glycoprotein,P04217,Secreted.,Extracellular (GO:0005576),3610142,secreted,100.0,0,secreted,secreted,Eutheria,Eutheria,Euteleostomi
1,A1CF,HGNC:24086,APOBEC1 complementation factor,Q9NQ94,Nucleus. Endoplasmic reticulum (By similarity)...,Nucleus (GO:0005634); Cytoplasm (GO:0005737),"10833526, 12881431, 12896982",nucleus,51.0,0,cytoplasm. nucleus,multiple,Deuterostomia,Eukaryota,Cell.Organisms
2,A2M,HGNC:7,alpha-2-macroglobulin,P01023,Secreted.,Extracellular (GO:0005576); Cytosol (GO:0005829),"9831625, 11823454, 15561729, 16188874, 16199891",secreted,100.0,0,cytoplasm. secreted,multiple,Euteleostomi,Deuterostomia,Cell.Organisms


,A,Aa,B,Bb,score,evidence
0,AL1A1_HUMAN,216,AL1A1_HUMAN,216,0.76,"experiments:in vivo,Two-hybrid;pmids:12081471,..."
1,ITA7_HUMAN,3679,ACHA_HUMAN,1134,0.73,"experiments:in vivo,Affinity Capture-Western,a..."
2,NEB1_HUMAN,55607,ACTG_HUMAN,71,0.65,"experiments:in vitro,in vivo;pmids:9362513,120..."


In [4]:
merged = pd.merge(uniprot_mapping, location_data, how='left', left_on='Entry', right_on='UniProtAC')
merged = merged[['Entry', 'Entry name', 'Protein names', 'Gene names', 'Consensus_SL']]
merged['Consensus_SL'] = merged['Consensus_SL'].str.replace('. ', ', ')
merged['Consensus_SL'] = '[' + merged['Consensus_SL'] + ']'

In [12]:
merged[:3]

,Entry,Entry name,Protein names,Gene names,Consensus_SL
0,P04217,A1BG_HUMAN,Alpha-1B-glycoprotein (Alpha-1-B glycoprotein),A1BG,secreted
1,Q9NQ94,A1CF_HUMAN,APOBEC1 complementation factor (APOBEC1-stimul...,A1CF ACF ASP,"cytoplasm, nucleus"
2,P01023,A2MG_HUMAN,Alpha-2-macroglobulin (Alpha-2-M) (C3 and PZP-...,A2M CPAMD5 FWP007,"cytoplasm, secreted"


In [70]:
hippie_merged = pd.merge(hippie, uniprot_mapping[['Entry name', 'Entry']], how='left', left_on='A', right_on='Entry name')
hippie_merged = pd.merge(hippie_merged, uniprot_mapping[['Entry name', 'Entry']], how='left', left_on='B', right_on='Entry name', suffixes=('_A','_B'))
hippie_merged = hippie_merged.dropna(subset=['Entry_A', 'Entry_B'])[['Entry_A', 'Entry_B', 'score']]

In [104]:
def find_interactions(mapping_row):
    ac = mapping_row['Entry']
    right = hippie_merged[hippie_merged['Entry_A'] == ac].values
    left = hippie_merged[hippie_merged['Entry_B'] == ac].values
    
    return [(x[1], x[2]) for x in right] + [(x[0], x[2]) for x in left]

In [109]:
merged['partners'] = merged.apply(find_interactions, axis=1)

In [115]:
merged.to_csv('data.csv', index=False)